In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import scale

import os 
os.getcwd()

'/tf/thesis_plastics'

In [8]:
data = pd.read_csv(os.getcwd() + '/data/data_reduced.csv')

In [ ]:
data.describe()

In [9]:
data.head()

,Matrix_family,Matrix_crystallinity,Matrix_topology,Matrix_SCB,Matrix_LCB,Matrix_viscosity,Contaminant_family,Contaminant_crystallinity,Contaminant_topology,Contaminant_SCB,...,MinorityPolymer_HDPE1,MinorityPolymer_LDPE1,MinorityPolymer_LLDPE1,MinorityPolymer_PA1,MinorityPolymer_PET1,MinorityPolymer_PP1,MinorityPolymer_PP2,MinorityPolymer_PP3,MinorityPolymer_PS1,MinorityPolymer_PS2
0,0,1,0,1,2,2,0,2,1,0,...,1,0,0,0,0,0,0,0,0,0
1,0,1,0,1,2,2,0,2,1,0,...,1,0,0,0,0,0,0,0,0,0
2,0,1,0,1,2,2,0,2,1,0,...,1,0,0,0,0,0,0,0,0,0
3,0,1,0,1,2,2,0,2,1,0,...,1,0,0,0,0,0,0,0,0,0
4,0,1,0,1,2,2,0,2,1,0,...,1,0,0,0,0,0,0,0,0,0


We have 328 instances with 47 features. HOW TO HANDLE THAT MANY ATTRIBUTES WITH SO LITTLE DATA

In [ ]:
data.head()

In [ ]:
LDPE1 = data[data['MajorityPolymer'] == 'LDPE1']

agg = data.groupby('MajorityPolymer').agg('mean')

LDPE1.head()

In [ ]:
LDPE1.head()
LDPE1_num = LDPE1[['WeightImpurity', 'POL', 'NOK', 'NOA','matrixE', 'matrixSTRENGTH', 'matrixStrainbreak' , 'matrixImpact', 'impurityE', 'impuritySTRENGTH', 'impurityStrainbreak', 'impurityImpact', 'matrixXc', 'impurityXc' , 'gammaABWu', 'Ddeltad2', 'Ddeltap2', 'Ddeltah2', 'blendE', 'blendSTRENGTH' ,'blendStrainbreak',
                'blendImpact','EblendOpEmatrix', 'STRENGTHblendOpSTRENGTHmatrix', 'StrainbreakblendOpStrainbreakmatrix', 'ImpactblendOpImpactmatrix'  ]] 
# hist= plt.hist(LDPE1_num)
# LDPE1_num.head()

In [ ]:
data['POL'].unique()

In [ ]:
q2house2 = data["STRENGTHblendOpSTRENGTHmatrix"][data["Matrix_family"] =='Polyolefin'].iloc[1].item()
plt.hist(q2house2)

In [ ]:
center_data = lambda x: x - x.mean()

to find the principal all categorical features must be encoded, for such we create dummy vars for the different polymers in the blend. 'HDPE1', 'LLDPE1', 'PP1', 'PP2', 'LDPE1', 'PP3', 'PET1', 'PS1',
       'PS2', 'PA1'
       
       
Matriix crystalinity can also be encoded theres low high and amorphous; ORDINAL?? 

matrix class? donnt know what it means, ['POL', 'POK', 'NOA', 'NOK'] same for contaminant,['POK', 'PURE', 'POL', 'NOK', 'NOA'], then theres class combo which is the combinatio of both of them so it will be droped

what are reeks??

Topology, 
PRROBLRMS:
SCB has none low high(0,1) and LCB has none and high(0,1,2). is crystalinity ordinal?. 

how to get the names of the components!!

**update 1/5 Anna: 

EblendOpEmatrix',
       'STRENGTHblendOpSTRENGTHmatrix',
       'StrainbreakblendOpStrainbreakmatrix', 'ImpactblendOpImpactmatrix'. such cols are ratios between the majority plastic and the blend given the feature. 
       
       
       'gammaABWu', 'Ddeltad2', 'Ddeltap2', 'Ddeltah2 cols are theoretical estimations that surge then teh classification of the families. thus cols Matrix Class','Impurity Class', 'Class Combo', 'POL', 'POK', 'NOK', 'NOA', are not used

In [ ]:
data['Impurity Class'].unique()

In [ ]:
data['Contaminant_viscosity'].unique()


In [ ]:
d = data
# ratio of polymerB 
d = d.replace({'impurityXc': {'PURE': 0}})

#POLYFINE VS NON POLYSINE 
d = d.replace({'Matrix_family': {'Polyolefin': 0, 'Non-Polyolefin': 1}})
d = d.replace({'Contaminant_family': {'Polyolefin': 0, 'Non-Polyolefin': 1}})

# Branched or linear topology 
d = d.replace({'Matrix_topology': {'Branched': 0, 'Linear': 1}})
d = d.replace({'Contaminant_topology': {'Branched': 0, 'Linear': 1}})

# crystlainity, amorphous shoudl be the lowest 
d = d.replace({'Matrix_crystallinity': {'Low': 1, 'Amorphous': 0, 'High': 2}})
d = d.replace({'Contaminant_crystallinity': {'Low': 1, 'Amorphous': 0, 'High': 2}})

# SCB
d = d.replace({'Matrix_SCB': {'None': 0, 'High': 1}})
d = d.replace({'Contaminant_SCB': {'None': 0, 'High': 1}})

# LCB
d = d.replace({'Matrix_LCB': {'None': 0, 'Low' :1, 'High': 2}})
d = d.replace({'Contaminant_LCB': {'None': 0, 'Low': 1, 'High': 2}})


# to be done,encode polymer types of blends and matrix family x2
d.columns.values

In [ ]:
# dropping series number and name and reeks
d = d.drop(['NAAM', 'SERIE', 'Reeks' ], axis=1)

# theoretical estimations
d = d.drop(['gammaABWu', 'Ddeltad2', 'Ddeltap2', 'Ddeltah2'], axis=1)
# Kims classes 
d = d.drop(['Matrix Class','Impurity Class', 'Class Combo', 'POL', 'POK', 'NOK', 'NOA'], axis=1) 
# ratios 
d = d.drop(['EblendOpEmatrix', 'STRENGTHblendOpSTRENGTHmatrix',
       'StrainbreakblendOpStrainbreakmatrix', 'ImpactblendOpImpactmatrix' ], axis=1)

d.columns.values

In [ ]:
# h1D = pd.get_dummies(d, columns = ['MajorityPolymer', 'MinorityPolymer', 'Matrix Class', 'Impurity Class'])
#only encode the type of polymer
h1D = pd.get_dummies(d, columns = ['MajorityPolymer', 'MinorityPolymer'])
new_df = h1D.select_dtypes(include=np.number)
new_df.columns.shape[0]

In [ ]:
h1D.columns.shape[0]

In [ ]:
# # SAVING DATA FILE
# h1D.to_csv("data_reduced.csv", index=False)

# PERFORMING PCA


In [ ]:
cor_mat2 = np.corrcoef(h1D.T)

eig_vals, eig_vecs = np.linalg.eig(cor_mat2)

In [ ]:
print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# # Visually confirm that the list is correctly sorted by decreasing eigenvalues
# print('Eigenvalues in descending order:')
# for i in eig_pairs:
#     print(i[0])


In [ ]:
# th
np.round(eig_vals**2 / sum(eig_vals**2), 2)
plt.plot(eig_vals**2);

In [ ]:
# cumulative PCA
tot = sum(eig_vals**2)
var_exp = [(i**2 / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

# plotting
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(6, 4))

    plt.bar(range(4), var_exp, alpha=0.5, align='center',
            label='individual explained variance')
    plt.step(range(4), cum_var_exp, where='mid',
             label='cumulative explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout()

In [ ]:

h1D.var()

In [ ]:
X = pd.DataFrame(scale(h1D), index=h1D.index, columns=h1D.columns)
pca_loadings = pd.DataFrame(PCA().fit(X).components_.T, index=h1D.columns, columns =h1D.columns.values )
d_plot = pd.DataFrame(PCA().fit_transform(X), index = X.index)


In [ ]:
fig , ax1 = plt.subplots(figsize=(9,7))

#ax1.set_xlim(-3.5,3.5)
#ax1.set_ylim(-3.5,3.5)

# Plot Principal Components 1 and 2
for i in d_plot.index:
    ax1.annotate(i, (d_plot.PC1.loc[i], -d_plot.PC2.loc[i]), ha='center')

# Plot reference lines
ax1.hlines(0,-3.5,3.5, linestyles='dotted', colors='grey')
ax1.vlines(0,-3.5,3.5, linestyles='dotted', colors='grey')

ax1.set_xlabel('First Principal Component')
ax1.set_ylabel('Second Principal Component')
    
# Plot Principal Component loading vectors, using a second y-axis.
ax2 = ax1.twinx().twiny() 

ax2.set_ylim(-1,1)
ax2.set_xlim(-1,1)
ax2.tick_params(axis='y', colors='orange')
ax2.set_xlabel('Principal Component loading vectors', color='orange')

# Plot labels for vectors. Variable 'a' is a small offset parameter to separate arrow tip and text.
a = 1.07  
for i in pca_loadings[['0', '1']].index:
    ax2.annotate(i, (pca_loadings.V1.loc[i]*a, -pca_loadings.V2.loc[i]*a), color='orange')

# Plot vectors
ax2.arrow(0,0,pca_loadings.V1[0], -pca_loadings.V2[0])
ax2.arrow(0,0,pca_loadings.V1[1], -pca_loadings.V2[1])
ax2.arrow(0,0,pca_loadings.V1[2], -pca_loadings.V2[2])
ax2.arrow(0,0,pca_loadings.V1[3], -pca_loadings.V2[3]);

In [ ]:
X_std = StandardScaler().fit_transform(X)
X_cnt = center_data(X)

# reducing to 2dim space , only choosoing the best 2 eigenvectors
# matrix_w = np.hstack((eig_pairs[0][1].reshape(59,1), 
#                       eig_pairs[1][1].reshape(59,1),
#                       eig_pairs[2][1].reshape(59,1),
#                       eig_pairs[3][1].reshape(59,1),
#                       eig_pairs[4][1].reshape(59,1)))
matrix_w = np.hstack((eig_pairs[0][1].reshape(68,1), 
                      eig_pairs[1][1].reshape(68,1),
                      eig_pairs[2][1].reshape(68,1),
                      eig_pairs[3][1].reshape(68,1),
                      eig_pairs[4][1].reshape(68,1)))


Y = X_std.dot(matrix_w)
Xprime = Y.dot(matrix_w.transpose())
np.square(Xprime-X_std).mean(axis=None)

In [ ]:
# OUTPUT PARAM! 
# blendE                                 8.227900e+05
# blendSTRENGTH                          2.744392e+02
# blendStrainbreak                       2.653426e+04
# blendImpact                            6.285922e+02
# EblendOpEmatrix                        1.796680e+00
# STRENGTHblendOpSTRENGTHmatrix          1.899984e-01
# StrainbreakblendOpStrainbreakmatrix    1.713696e+02
# ImpactblendOpImpactmatrix              1.212540e+01

dT = h1D.drop(['blendE','blendSTRENGTH','blendStrainbreak', 'blendImpact','EblendOpEmatrix','STRENGTHblendOpSTRENGTHmatrix', 'StrainbreakblendOpStrainbreakmatrix', 'ImpactblendOpImpactmatrix'], axis=1)
#  d.drop(['Class Combo','NAAM', 'SERIE' ], axis=1)
yT = h1D[['blendE','blendSTRENGTH','blendStrainbreak', 'blendImpact','EblendOpEmatrix','STRENGTHblendOpSTRENGTHmatrix', 'StrainbreakblendOpStrainbreakmatrix', 'ImpactblendOpImpactmatrix']]

In [ ]:
DT = pd.DataFrame(scale(dT), index=dT.index, columns=dT.columns)
DT = StandardScaler().fit_transform(DT)
DT = center_data(DT)

cor_mat2 = np.corrcoef(DT.T)
eig_vals, eig_vecs = np.linalg.eig(cor_mat2)
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# reducing to 2dim space , only choosoing the best 4 eigenvectors
matrix_w = np.hstack((eig_pairs[0][1].reshape(60,1), 
                      eig_pairs[1][1].reshape(60,1),
                      eig_pairs[2][1].reshape(60,1),
                      eig_pairs[3][1].reshape(60,1),
                      eig_pairs[4][1].reshape(60,1)))

Y = DT.dot(matrix_w)
DTprime = Y.dot(matrix_w.transpose())
np.square(DTprime-DT).mean(axis=None)

# families

In [ ]:
dataF = pd.read_table("C:/Users/33789/OneDrive/Desktop/THESIS/data/data_families.csv", ',')


In [ ]:
dataF.columns.values

In [ ]:
POK = data[data['Matrix Class'] == 'POK']
NOK = data[data['Matrix Class'] == 'NOK']
POL = data[data['Matrix Class'] == 'POL']
NOA = data[data['Matrix Class'] == 'NOA']

In [ ]:
sns.relplot(
    data=data, x='WeightImpurity', y='blendSTRENGTH',
    col="Matrix Class", hue="Impurity Class", 
    kind="line"
)

In [ ]:
sns.relplot(
    data=data, x='WeightImpurity', y='blendE',
    col="Matrix Class", hue="Impurity Class", 
    kind="line"
)

In [ ]:
sns.relplot(
    data=data, x='WeightImpurity', y='blendStrainbreak',
    col="Matrix Class", hue="Impurity Class", 
    kind="line"
)

#        'blendSTRENGTH', 'blendStrainbreak', 'blendImpact',

In [ ]:
sns.relplot(
    data=data, x='WeightImpurity', y='blendImpact',
    col="Matrix Class", hue="Impurity Class", 
    kind="line"
)

# Gaussian Process

The idea would be to given the majority and minority function of chnage in given property. 
to min just looking at strength of features;
STRENGTH
then try the same but not by polymers but families of DR KIM

In [ ]:
# A: LDPE B: HDPE STRENGTH
# not using one hotenconded data
LDPE =d[d['MajorityPolymer']== 'LDPE1']
LPDE_HDPE=LDPE[ LDPE['MinorityPolymer']== 'HDPE1']
LPDE_HDPET=LPDE_HDPE.iloc[:,2:15]
# 16,17 matrix,impurity class; taki,g the order frommatlab code
# 21matrix E, 25: I E. 35 E blend= y 
x = pd.concat([LPDE_HDPET,LPDE_HDPE.iloc[:,21], LPDE_HDPE.iloc[:,25]], axis=1)
y = LPDE_HDPE.iloc[:,35]
x.head()


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
clf = GaussianProcessRegressor()
clf.fit(x, y)
y_pred = clf.predict(x)

In [ ]:
plt.plot(x['WeightImpurity'], y, marker='o', color='blue')
plt.plot(x['WeightImpurity'], y_pred, marker= 'x',color = 'red')
plt.xlabel("ratio impurity")
plt.ylabel("Elasticity")
plt.show()

### trying with POL (A) and B: POK 



In [ ]:

POL =h1D[h1D['Matrix Class_POL']== 1]
POL_POK=POL[POL['Impurity Class_POK']== 1]
POL_POKT=POL_POK.iloc[:,2:15]
xF = pd.concat([POL_POKT,POL_POK.iloc[:,21], POL_POK.iloc[:,25]], axis=1)
yF = POL_POK.iloc[:,35]
sns.histplot( POL_POK['blendE'])
sns.histplot(LPDE_HDPE['blendE'])
# POL.head()

In [ ]:

clf = GaussianProcessRegressor()
clf.fit(xF, yF)
y_pred = clf.predict(xF)
plt.plot(xF['WeightImpurity'], yF, marker='o', color='blue')
plt.plot(xF['WeightImpurity'], y_pred, marker= 'x',color = 'red', linewidth=1)
plt.xlabel("ratio impurity")
plt.ylabel("Elasticity")
plt.title('Elasticity of POL vs POK')
plt.show()

In [ ]:
# LDEP is POK, tryin with HDPE1_PET1 that in NOK
HDPE =d[d['MajorityPolymer']== 'HDPE1']
HPDE_PET=HDPE[ HDPE['MinorityPolymer']== 'PET1']
# training features
HPDE_PETT=HPDE_PET.iloc[:,2:15]
xF = pd.concat([HPDE_PETT,HPDE_PET.iloc[:,21], HPDE_PET.iloc[:,25]], axis=1)
yF = HPDE_PET.iloc[:,35]


In [ ]:
clf = GaussianProcessRegressor()
clf.fit(x, y)
y_pred = clf.predict(x)
plt.plot(x['WeightImpurity'], y, marker='o', color='blue')
plt.plot(x['WeightImpurity'], y_pred, marker= 'x',color = 'red')
plt.xlabel("ratio impurity")
plt.ylabel("Elasticity")
plt.title('Elasticity of HPDE_PET')
plt.show()

# SVMs


In [ ]:
x_e = h1D.drop(['matrixSTRENGTH', 'matrixStrainbreak',
       'matrixImpact','impuritySTRENGTH',
       'impurityStrainbreak', 'impurityImpact', 'blendE', 'blendSTRENGTH', 'blendStrainbreak', 'blendImpact'], axis =1)
y_e = h1D['blendE']
X_train, X_test, y_train, y_test = train_test_split(x_e, y_e, test_size = 0.2, random_state = 0)

Such a grouping of data is domain specific. thus IID assumptioon is broken 

In [ ]:
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# def print_score(clf, X_train, y_train, X_test, y_test, train=True):
#     if train:
#         pred = clf.predict(X_train)
#         clf_report = pd.DataFrame(r2_score(y_true, y_pred))
#         print("Train Result:\n================================================")
#         print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
#         print("_______________________________________________")
#         print(f"CLASSIFICATION REPORT:\n{clf_report}")
#         print("_______________________________________________")
#         print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
#     elif train==False:
#         pred = clf.predict(X_test)
#         clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
#         print("Test Result:\n================================================")        
#         print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
#         print("_______________________________________________")
#         print(f"CLASSIFICATION REPORT:\n{clf_report}")
#         print("_______________________________________________")
#         print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [ ]:
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
for train_index, test_index in ss.split(x_e):
    print("%s %s" % (train_index, test_index))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from tqdm import tqdm_notebook as tqdm


standardized_data = (h1D - h1D.mean())/ h1D.std()
x_e = standardized_data.drop(['matrixSTRENGTH', 'matrixStrainbreak',
       'matrixImpact','impuritySTRENGTH',
       'impurityStrainbreak', 'impurityImpact', 'blendE', 'blendSTRENGTH', 'blendStrainbreak', 'blendImpact'], axis =1)
y_e = h1D['blendE']
X_train, X_test, y_train, y_test = train_test_split(x_e, y_e, test_size = 0.2, random_state = 0)



model = SVR(kernel='rbf')
model.fit(X_train, y_train)
# print_score(model, X_train, y_train, X_test, y_test, train=True)
# print_score(model, X_train, y_train, X_test, y_test, train=False)

pipe_steps= [('SVM', SVR())]

#TODO: doo same with gaussian
params= {
    'SVM__C': [0.01, 0.1, 0.5, 1, 10, 100],
    'SVM__kernel':['rbf', 'poly', 'linear'],
    'SVM__gamma' :[50 ,10 ,5 , 1, 0.75, 0.5, 0.25, 0.1, 0.01, 0.001]
}
pipeline= Pipeline(pipe_steps)
# param_grid = {'C': [0.01, 0.1, 0.5, 1, 10, 100], 
#               'gamma': [1, 0.75, 0.5, 0.25, 0.1, 0.01, 0.001], 
#               'kernel': ['rbf', 'poly', 'linear']} 

#grid = GridSearchCV(SVR(), param_grid, refit=True, verbose=1, cv=5)
#grid.fit(X_train, y_train)


In [ ]:
# #4 CV 
# for cv in range(4,6):
#     print('entering : -----------------------------------')
#     print(cv)
#     grid = GridSearchCV(pipeline, param_grid = params, cv=cv)
#     grid.fit(X_train, y_train)
#     print('scoreefor %d fold CV = %3.2f' %(cv, grid.score(X_test, y_test)))
#     print('best param from TD')
#     print(grid.best_params_)
    

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline



# param_grid = [
#   {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'kernel': ['linear']},
#   {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'gamma': [1,0.001, 0.0001], 'kernel': ['rbf']},
#   {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'kernel': ['poly'],'degree':[0, 2, 4, 6] },
#  ]
param_grid = [{
            'C': [0.001, 0.01, 1, 10], 
            #   'gamma': [1, 0.75, 0.5, 0.25, 0.1, 0.01, 0.001], 
            'gamma': [1,  0.1, 0.001], 
            'kernel': ['rbf', 'poly', 'linear']
              } ]


svm = SVR()
grid = GridSearchCV(svm, param_grid, cv = 3, scoring = 'neg_mean_absolute_percentage_error', return_train_score = True, verbose = 10 )
print('grid done')
grid.fit(X_train, y_train)
print('fitting done')

# n_components = grid.cv_results_["C"]
# test_scores = n_components n_components n_components n_components n_components n_components "kernel"]


In [ ]:
from sklearn import metrics
sorted(metrics.SCORERS.keys())

In [ ]:
n_components 

In [ ]:

rbf_svr = svm.SVR(kernel='rbf')
rbf_svr.fit(x_e, y_e)
y_predF = rbf_svr.predict(xF)
plt.plot(x_e['WeightImpurity'], y_e, marker='o', color='blue')
plt.plot(x['WeightImpurity'], y_predF, marker= 'x',color = 'red')
plt.xlabel("ratio impurity")
plt.ylabel("Elasticity")
plt.title('Elasticity of HPDE_PET')
plt.show()

DESCRIPTORS OF CHEMISTRY IN ML CAN BE SUPER POWERFUL AND CAN BE ASSESSED BY THEIR IMPORTANCE? THEY CAN ENCAPSULATE NONLINEAR BEHAVOIRS AND PATTERNS IN DATA 
THEY ARE KNWOLEDGE DRIVEN AND THEN TESTED. 
FOR THIS EXPERIMENT THAT WE HAVE DIFFERENT FAMILIES AND PROPERTIES TO PREDUCT? IT MAY BE EASIER TO COUNT WITH THEM 
ELSE KERNEL RIDGE REGRESSION WILL BE NEXT STEP. 
KERNELSCAN BE SIGMMOID? POLYN? LINEAR? HOW TO FIND THE MOST SUITABLE, 

# filtering

In [ ]:
# starting with new h1D that was redecued alread, first apply contsnat rep$moval variables of variance 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold

In [ ]:
# only with ELASTICITY, 14? 18? 24 

In [ ]:
x_e = h1D.drop(['matrixSTRENGTH', 'matrixStrainbreak',
       'matrixImpact','impuritySTRENGTH',
       'impurityStrainbreak', 'impurityImpact', 'blendE', 'blendSTRENGTH', 'blendStrainbreak', 'blendImpact'], axis =1)
y_e = h1D['blendE']
      

In [ ]:
# forthe moment stratify in E to insure that all values of E are in all sets 
X_train, X_test, y_train, y_test = train_test_split(x_e, y_e, test_size = 0.2, random_state = 0)

In [ ]:
X_test, y_train, y_test = train_test_split(x_e, y_e, test_size = 0.2, random_state = 0)

In [ ]:
# trying for QUASIconstant filter because constant returned 0
constant_filter = VarianceThreshold(threshold=0.1)
constant_filter.fit(X_train)
VarianceThreshold(threshold=0.1)

In [ ]:
constant_filter.get_support().sum()
constant_list = [not temp for temp in constant_filter.get_support()]
x_e.columns[constant_list]
# NONE!!

whne threshold =0.1, these are the cols quasi constant: 'WeightImpurity', 'MajorityPolymer_PA1', 'MajorityPolymer_PET1',
       'MajorityPolymer_PP1', 'MajorityPolymer_PP2', 'MajorityPolymer_PP3',
       'MajorityPolymer_PS2', 'MinorityPolymer_LLDPE1', 'MinorityPolymer_PA1',
       'MinorityPolymer_PP2', 'MinorityPolymer_PP3', 'MinorityPolymer_PS2'
       
       meaning the ratio of impurity; and the Mayority and minority polymer are not that relevant!

In [ ]:
X_train_filter = constant_filter.transform(X_train)
X_test_filter = constant_filter.transform(X_test)
X_train_filter.shape, X_test_filter.shape, X_train.shape

**Pearsons Correlation
from filetring apply corr for every col with target and find most avalauble 

In [ ]:
from scipy.stats import pearsonr